# import libraries

In [10]:
import sys
import os
os.chdir('/Users/serdarturkaslan/Documents/GitHub/GbmMINER/data/MINER_MicroLowessRNATMM.08.24.2020/')
#import miner as miner
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
import seaborn as sns
import time
import imp
imp.reload(miner)
import miner_py3_kk as miner

# Create directories to save output file

In [11]:
# Path to the miner directory
input_path = os.getcwd()
print(input_path)

# create name for results folder where output files will be saved
resultsFolder = "results_miner3_gbm_MicroarrayRNA_survival_08112021"

# name of the folder containing the miner network
#networkFolder = "miner_network_results"
networkFolder = "results_minCorrelation_0o2_50_allFiles_gbm_MicroarrayRNA_survival_08112021"

# create results directory
resultsDirectory = os.path.join(os.getcwd(),resultsFolder)
if not os.path.isdir(resultsDirectory):
    os.mkdir(resultsDirectory)

expressionData=pd.read_csv("GbmMicroRNAMergedWithIDsZScored.csv",header=0,index_col=0,sep=",")


/Users/serdarturkaslan/Documents/GitHub/GbmMINER/data/MINER_MicroLowessRNATMM.08.24.2020


# Load primary data

In [12]:
regulonModules = miner.read_json(os.path.join(input_path,"regulons.json"))

# Load primary survival data

In [13]:
# load primary survival data (i.e., corresponding to expression data training set)
survivalMMRF = pd.read_csv(os.path.join(input_path,"TCGA_Survival_Gbm.csv"),index_col=0,header=0)
survivalDfMMRF = survivalMMRF.iloc[:,0:2]
survivalDfMMRF.columns = ["duration","observed"]
kmDfMMRF = miner.kmAnalysis(survivalDf=survivalDfMMRF,durationCol="duration",statusCol="observed")
guanSurvivalDfMMRF= miner.guanRank(kmSurvival=kmDfMMRF)

# Survival analysis of regulons

In [14]:
t1 = time.time()

cox_regulons_output = miner.parallelMedianSurvivalAnalysis(regulonModules,expressionData,guanSurvivalDfMMRF,numCores=5)
cox_regulons_output = cox_regulons_output.iloc[np.argsort(np.array(cox_regulons_output.index).astype(int))]
cox_regulons_output.to_csv(os.path.join(resultsDirectory,'CoxProportionalHazardsRegulons.csv'))
cox_regulons_output.sort_values(by="HR",ascending=False,inplace=True)

print("\nHigh-risk regulons:")
print(cox_regulons_output.iloc[0:5,:])
print("\nLow-risk regulons")
print(cox_regulons_output.iloc[-5:,:])

t2 = time.time()
print("Completed regulon risk analysis in {:.2f} minutes".format((t2-t1)/60.))

Completed 0 of 752 iterations
Completed 0 of 753 iterations
Completed 0 of 753 iterations
Completed 0 of 753 iterations
Completed 0 of 753 iterations
Completed 100 of 753 iterations
Completed 100 of 752 iterations
Completed 100 of 753 iterations
Completed 100 of 753 iterations
Completed 100 of 753 iterations
Completed 200 of 752 iterations
Completed 200 of 753 iterations
Completed 200 of 753 iterations
Completed 200 of 753 iterations
Completed 200 of 753 iterations
Completed 300 of 753 iterations
Completed 300 of 753 iterations
Completed 300 of 753 iterations
Completed 300 of 753 iterations
Completed 300 of 752 iterations
Completed 400 of 753 iterations
Completed 400 of 753 iterations
Completed 400 of 753 iterationsCompleted 400 of 753 iterations

Completed 400 of 752 iterations
Completed 500 of 753 iterations
Completed 500 of 753 iterations
Completed 500 of 753 iterations
Completed 500 of 753 iterations
Completed 500 of 752 iterations
Completed 600 of 753 iterations
Completed 600 of 7